In [190]:

import os

from collections import Counter

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import yaml
import re

from plotly.figure_factory import create_scatterplotmatrix
from plotly.subplots import make_subplots
import plotly.express as px 
import plotly.graph_objects as go

import torch
from konlpy.tag import Okt
from transformers import AutoTokenizer
from itertools import chain
from scipy import stats

pd.options.display.float_format = '{:.2f}'.format

## 빈출 단어 구하기, 토크나이저 초기화

In [191]:
train_df = pd.read_csv('../NLP_dataset/train.csv')
test_df = pd.read_csv('../NLP_dataset/test.csv')
dev_df = pd.read_csv('../NLP_dataset/dev.csv')

with open('../sbert_config.yaml', "r") as f:
    config = yaml.load(f, Loader=yaml.Loader)
tokenizer = AutoTokenizer.from_pretrained(config['base_model'])

word_bags_n1 = []
word_bags_n1 += chain(*[tokenizer.encode(s) for s in train_df['sentence_1']])
word_bags_n1 += chain(*[tokenizer.encode(s) for s in train_df['sentence_2']])
word_bags_n1 += chain(*[tokenizer.encode(s) for s in dev_df['sentence_1']])
word_bags_n1 += chain(*[tokenizer.encode(s) for s in dev_df['sentence_2']])
word_bags_n1 = Counter(word_bags_n1)

## plot용 함수

In [202]:
def label_scatter_plot(df, color_col, vs_cols, hover_col=None, size = (800, 800)):

    fig = px.scatter(df, 
                    x=vs_cols[0],
                    y=vs_cols[1],
                    color=color_col,
                    hover_data=hover_col,
                    width=size[0],
                    height=size[1],
                    color_continuous_scale=px.colors.sequential.Jet
                    
    )
    fig.show()
    
def get_category_index(fd, REFER_COL, TH_LIST=None):
    if TH_LIST is None:
        # low outlier, low outlier~25%, 25~75%, 75~high outlier, high outlier~
        TH_LIST = [0, 0, 0, 0]
        TH_LIST[1] = fd[REFER_COL].quantile(0.25)
        TH_LIST[2] = fd[REFER_COL].quantile(0.75)
        IQR = TH_LIST[2] - TH_LIST[1]
        # NOTE : BoxPlot의 outlier는 IQR*1.5를 더하거나 뺀 값보다 큰 최소, 최대값이 TH이 되나, 본 코드에서는 편의상 생략.
        TH_LIST[3] = TH_LIST[2] + IQR*1.5
        TH_LIST[0] = TH_LIST[1] - IQR*1.5

    index = [0]*(len(TH_LIST)+1)
    for i in range(len(TH_LIST)+1):
        if i == 0:
            index[i] = fd[REFER_COL] < TH_LIST[i] 
        elif i == len(TH_LIST):
            index[i] = fd[REFER_COL] >= TH_LIST[i-1]
        else:
            index[i] = (fd[REFER_COL] >= TH_LIST[i-1]) & (fd[REFER_COL] < TH_LIST[i])
    
    return index, TH_LIST

def show_violin_plot(fd, REFER_COL, CHECK_COL, SHOW_VIOLIN, TH_LIST=None, size = (500, 1000)):
    index, TH_LIST = get_category_index(fd, REFER_COL, TH_LIST)
    
    fig = make_subplots(
        rows=1, cols=len(TH_LIST)+1,
        subplot_titles=([f'~{TH_LIST[0]:.2f}'] + list(map(lambda x: f'{x:.2f}', TH_LIST)))
    )
 
    if SHOW_VIOLIN:
        for i in range(len(TH_LIST)+1):
            fig.add_trace(go.Violin(y = fd[CHECK_COL][index[i]], points='all'), row=1, col=i+1)
    else:
        for i in range(len(TH_LIST)+1):
            fig.add_trace(go.Box(y = fd[CHECK_COL][index[i]]), row=1, col=i+1)

    fig.update_layout(height=size[0], width=size[1],
                    title_text= f"{CHECK_COL}, ref:{REFER_COL}")

    fig.show()

## feature engineering func
- 보고 싶은 feature가 있을 경우 아래에 추가하면 됩니다. (코사인 유사도 등)

In [207]:

# 아래 col이 반드시 포함되어야 함.
# pred: float
# label: float
# s1, s2: text

# 이외의 col들은 자유롭게 추가가능.
NECESSARY_COL = set(['pred', 'label', 's1', 's2'])

STOPWORDS = pd.read_csv('stopwords.csv')
STOPWORDS = set(STOPWORDS['word'])

    
def feature_engineering(df, tokenizer):
    okt = Okt()
    aux_features = [col for col in df.columns if not col in NECESSARY_COL]
    
    # 라벨차이     
    df['diff'] = abs(df['label'] - df['pred'])
    
    s1 = [re.sub(' \[PAD\]', "", row['s1']).strip() for i, row in df.iterrows()]
    s2 = [re.sub(' \[PAD\]', "", row['s2']).strip() for i, row in df.iterrows()]
    
    # 공백 개수
    df['s1_space'] = [s.count(' ') for s in s1]
    df['s2_space'] = [s.count(' ') for s in s2]
    
    # 문장 길이
    df['s1_len'] = [len(s) for s in s1]
    df['s2_len'] = [len(s) for s in s2]
    
    # 불용어 개수
    df['s1_stop'] = [len([w for w in okt.morphs(s) if w in STOPWORDS]) for s in s1]
    df['s2_stop'] = [len([w for w in okt.morphs(s) if w in STOPWORDS]) for s in s2]
    
    # 토큰 개수
    df['s1_token'] = [len(tokenizer.encode(s)) for s in s1]
    df['s2_token'] = [len(tokenizer.encode(s)) for s in s2]
    
    # 빈도의 역수
    df['s1_rfreq'] = [sum(1/word_bags_n1[w] for w in tokenizer.encode(s))/len(s) for s in s1]
    df['s2_rfreq'] = [sum(1/word_bags_n1[w] for w in tokenizer.encode(s))/len(s) for s in s1]
    
    return df

## sample feature engineering func
- 이 함수는 표본집단에 대한 통계값을 데이터값으로 사용합니다. 예를 들어 피어슨 상관계수의 경우 샘플 하나에 대해서 계산할 수 없고 여러 개의 샘플(표본집단)에 대해서 계산이 가능합니다. 표본 집단을 나누는 기준은 reference col입니다.
- 이 함수로 반환된 데이터프레임은 feature engineering func에서 반환된 데이터프레임과 동일하게 사용할 수 있습니다.

In [208]:
def sample_feature_engineering(df, refer_col, check_col, th_list=None):
    index_list, th_list = get_category_index(df, refer_col, th_list)
    
    sfd = pd.DataFrame()
    for idx, th in zip(index_list, [f'~{th_list[0]}'] + th_list):
        if df[:][idx].shape[0] == 0:
            print(f'{th} : no data.')
            continue
        row = dict()
        for col in check_col:
            if not isinstance(col, tuple):
                # 일변량 통계값
                row[col + '_mean'] = [df[col][idx].mean()]
                row[col + '_std'] = [df[col][idx].mean()]
            else:
                # 다변량(2) 통계값
                a, b = col
                pearson = PearsonCorrCoef()
                row[f'{a}_{b}_pearson'] = float(pearson(torch.tensor(df[a][idx].values), 
                                                torch.tensor(df[b][idx].values)))     
        # refer_col를 카테고리로 보면 됨.
        row[refer_col] = th
        sfd = pd.concat([sfd, pd.DataFrame(row)], ignore_index=True)

    return sfd

# 1. feature들의 mean, std, 4분위 통계량 확인
- 결과를 읽고 feature engineering한 dataframe를 fd에 반환합니다.

In [209]:
data = pd.read_csv(f'output/BERT_3_0.8804768919944763.csv')
fd = feature_engineering(data, tokenizer)
print(fd.describe())


       Unnamed: 0  label   pred    rtt  source   diff  s1_space  s2_space  \
count      550.00 550.00 550.00 550.00  550.00 550.00    550.00    550.00   
mean       274.50   2.58   2.35   0.36    1.01   0.57      5.61      4.97   
std        158.92   1.46   1.36   0.48    0.81   0.46      3.80      3.92   
min          0.00   0.00  -0.11   0.00    0.00   0.00      1.00      0.00   
25%        137.25   1.40   1.07   0.00    0.00   0.22      3.00      2.00   
50%        274.50   2.60   2.42   0.00    1.00   0.47      4.00      4.00   
75%        411.75   3.80   3.66   1.00    2.00   0.81      7.00      6.00   
max        549.00   5.00   4.44   1.00    2.00   2.86     24.00     29.00   

       s1_len  s2_len  s1_stop  s2_stop  s1_token  s2_token  s1_rfreq  \
count  550.00  550.00   550.00   550.00    550.00    550.00    550.00   
mean    29.64   24.56     1.68     1.73     15.71     14.90      0.02   
std     15.66   14.96     1.96     1.98      8.40      7.64      0.02   
min     16.00 

In [210]:
refer_cols = 'diff'
check_cols = ['pred', 'label', ('pred', 'label')]
# th_list가 None일 경우 사분위 수를 기준으로 자동으로 카테고리화 함
th_list = None # [0.5, 1, 1.5, 2]
sfd = sample_feature_engineering(fd, refer_cols, check_cols, th_list)
print(sfd)
print(sfd.describe())

~-0.6653274707496166 : no data.
   pred_mean  pred_std  label_mean  label_std  pred_label_pearson  diff
0       2.07      2.07        2.08       2.08                1.00 -0.67
1       2.57      2.57        2.75       2.75                0.95  0.22
2       2.25      2.25        2.72       2.72                0.56  0.81
3       1.65      1.65        3.05       3.05               -0.11  1.70
       pred_mean  pred_std  label_mean  label_std  pred_label_pearson  diff
count       4.00      4.00        4.00       4.00                4.00  4.00
mean        2.14      2.14        2.65       2.65                0.60  0.52
std         0.39      0.39        0.41       0.41                0.51  1.00
min         1.65      1.65        2.08       2.08               -0.11 -0.67
25%         1.96      1.96        2.56       2.56                0.39 -0.00
50%         2.16      2.16        2.74       2.74                0.75  0.52
75%         2.33      2.33        2.83       2.83                0.96  1.03


# 2. scatter plot
- 관심있는 두 지표간의 상관 관계를 볼 수 있으며, color map을 통해 제 3의 지표간 관계도 같이 확인이 가능합니다.
- 또는 여러 지표에서 두 지표를 골라 상관관계를 볼 수있는 scatter plot matrix를 지원합니다.

In [211]:
# fd 데이터프레임에 대해 scatter plot
# 관심있는 col 정보 입력 
color_col = 'diff'
vs_col = ['pred', 'label']
# dot에 마우스를 가져다 될 때 뜨는 정보 확인
hover_col = ['s1', 's2', 's1_token', 's1_stop', 's1_space', 's1_rfreq', 'rtt', 'source']
label_scatter_plot(fd, color_col, vs_col, hover_col)

In [206]:
# sfd 데이터프레임에 대해 scatter plot
# 관심있는 col 정보 입력 
color_col = 'diff'
vs_col = ['diff', 'pred_label_pearson']
# dot에 마우스를 가져다 될 때 뜨는 정보 확인
hover_col = ['label_mean', 'label_std', 'pred_mean', 'pred_std']
label_scatter_plot(sfd, color_col, vs_col, hover_col, size=(500, 500))

In [212]:
# pair plot 열 정보 입력
index_col = 'rtt'
matrix_check_col = ['label','diff', 's1_rfreq', 'rtt']

fig = create_scatterplotmatrix(fd[matrix_check_col], 
                               diag = 'histogram',
                               width = 1000, 
                               height = 1000,
                               index= index_col,
                               colormap_type = 'cat'                         
                               )
fig.show()

# 2. box, violin plot
- 전체 데이터를 reference data에 따라 그룹화합니다.
- 각 그룹별로 box plot 또는 violin plot을 확인할 수 있습니다.

In [81]:
# 실수형 데이터를 백분위별로 나누어 Reference하는 예시

REFER_COL = "diff" 
CHECK_COL = ['label', 'pred', 's1_fre', 's1_len', 's1_token', 's1_space', 's1_stop'] 

TH_LIST =  None
# box plot으로 할 경우 False
SHOW_VIOLIN = False
for c in CHECK_COL:
    show_violin_plot(fd, REFER_COL, c, SHOW_VIOLIN, TH_LIST)

In [82]:
# 범주형 데이터를 Reference하는 예시

REFER_COL = "source" 
CHECK_COL = ['label', 'pred', 's1_rfreq', 's1_len', 's1_token', 's1_space', 's1_stop'] 

TH_LIST =  [0.5, 1.5]
# box plot으로 할 경우 False
SHOW_VIOLIN = True
for c in CHECK_COL:
    show_violin_plot(fd, REFER_COL, c, SHOW_VIOLIN, TH_LIST)

In [173]:
print('\033[95m' + '안녕' + '\033[96m' + '하세요' + '\033[0m')

안녕하세요
